# ISL Dataprep
## Run Mediapipe on downsampled and trimmed videos to generate .npz output

In [1]:
!nproc

44


## Set up data

Data Source: Lambda filesystem
  - bucket name: 921c05b3-8e02-4c0b-9623-d1de39939163
  - region: us-east-3


AWS creds needed:
  - AWS_ACCESS_KEY_ID=9P7Q2ADQR5WPGY83629B
  - AWS_SECRET_ACCESS_KEY=*********
  - AWS_REGION=us-east-3
  - S3_ENDPOINT_URL="https://files.us-east-3.lambda.ai"

Change the bucket and credentials when using a different instance. These are given as pattern reference.

In [ ]:
!pip install awscli boto3

In [3]:
! aws --version

aws-cli/1.42.36 Python/3.12.11 Linux/6.6.97+ botocore/1.40.36


In [4]:
import os
os.environ['AWS_ACCESS_KEY_ID'] = 'C0INV916QMS8WB9F104E'
os.environ['AWS_SECRET_ACCESS_KEY'] = '**********'
os.environ['AWS_DEFAULT_REGION'] = 'us-east-3'

In [ ]:
! aws configure

In [6]:
!echo "[default]" > ~/.aws/config
!echo "region =us-east-3 " >> ~/.aws/config
!echo "endpoint-url =https://files.us-east-3.lambda.ai " >> ~/.aws/config
!echo "" >> ~/.aws/config


!echo "request_checksum_calculation =when_required " >> ~/.aws/config
!echo "response_checksum_validation =when_required " >> ~/.aws/config
!echo "" >> ~/.aws/config


In [7]:
cat ~/.aws/config

[default]
region =us-east-3 
endpoint-url =https://files.us-east-3.lambda.ai 

request_checksum_calculation =when_required 
response_checksum_validation =when_required 



In [8]:
!aws s3 ls s3://6a8e9853-8a1d-4662-9fa6-be70cced7c34/outputs/ --endpoint-url https://files.us-east-3.lambda.ai | wc

   4350   17400  210540


In [ ]:
!aws s3 cp s3://6a8e9853-8a1d-4662-9fa6-be70cced7c34/outputs/ /content/isl_gospel_videos --recursive --exclude "*" --include "*.mp4" --endpoint-url https://files.us-east-3.lambda.ai

In [10]:
! df -h /

Filesystem      Size  Used Avail Use% Mounted on
overlay         226G   67G  160G  30% /


In [11]:
ls /content/

isl_gospel_videos/  sample_data/


## Set up code

In [12]:
! mkdir /content/logs

In [ ]:
!apt-get update && apt-get install -y parallel

In [14]:
!parallel --version

GNU parallel 20210822
Copyright (C) 2007-2021 Ole Tange, http://ole.tange.dk and Free Software
Foundation, Inc.
License GPLv3+: GNU GPL version 3 or later <https://gnu.org/licenses/gpl.html>
This is free software: you are free to change and redistribute it.
GNU parallel comes with no warranty.

Web site: https://www.gnu.org/software/parallel

When using programs that use GNU Parallel to process data for publication
please cite as described in the manpage.


In [ ]:
! git clone https://github.com/kavitharaju/sign-bibles-dataset.git

In [16]:
cd /content/sign-bibles-dataset/dataprep/isl/

/content/sign-bibles-dataset/dataprep/isl


In [ ]:
! ls /content/sign-bibles-dataset/dataprep/isl

In [18]:
!git checkout -b split-cpu-pipeline-into-local-n-cloud

Switched to a new branch 'split-cpu-pipeline-into-local-n-cloud'


In [ ]:
!git pull origin split-cpu-pipeline-into-local-n-cloud

In [20]:
! git status

On branch split-cpu-pipeline-into-local-n-cloud
nothing to commit, working tree clean


In [ ]:
! pip install -r requirements_mp.txt

## Run data processing

In [23]:
!ls /content/isl_gospel_videos/*.mp4 | xargs -n 1 basename -s .mp4 > /content/full_input_list.txt

In [41]:
! sed -n '601,870p' /content/full_input_list.txt > /content/input_list.txt

In [ ]:
!time(bash run_parallel_mp.sh)

In [ ]:
cat /content/logs/success.log

## Upload outputs

In [ ]:
pwd

'/content/sign-bibles-dataset/dataprep/isl'

In [43]:
! ls /content/isl_gospel_videos/*.npz | wc

    870     870   44370


In [ ]:
! aws s3 sync /content/isl_gospel_videos/ s3://6a8e9853-8a1d-4662-9fa6-be70cced7c34/outputs/ --endpoint-url=https://files.us-east-3.lambda.ai  --exclude "*" --include "*.npz"

In [45]:
! aws s3 ls s3://6a8e9853-8a1d-4662-9fa6-be70cced7c34/outputs/ --endpoint-url=https://files.us-east-3.lambda.ai | grep mediapipe.npz | wc

    870    3480   47850
